In [1]:
import requests 

url = "https://apis.data.go.kr/B551011/GoCamping/basedList"
requests_params = {
    'numOfRows':'50',
    'pageNo':'1',
    'MobileOS':'ETC',
    'MobileApp':'camping',
    'serviceKey':'9CddWKBqyjQQ6afe3WLOVYPvCrXYuxPPd3%2BeA%2F7mVVSdg%2B2%2BZJXGyC%2B0RwPk8wDMSQpJyHc1gXYzC3h4OErgiQ%3D%3D',
    '_type':'xml'
}
xml_data=xml.
res = requests.get(url,params=requests_params)
print(res)

SSLError: HTTPSConnectionPool(host='apis.data.go.kr', port=443): Max retries exceeded with url: /B551011/GoCamping/basedList?numOfRows=50&pageNo=1&MobileOS=ETC&MobileApp=camping&serviceKey=9CddWKBqyjQQ6afe3WLOVYPvCrXYuxPPd3%252BeA%252F7mVVSdg%252B2%252BZJXGyC%252B0RwPk8wDMSQpJyHc1gXYzC3h4OErgiQ%253D%253D&_type=xml (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_ILLEGAL_PARAMETER] sslv3 alert illegal parameter (_ssl.c:1028)')))

In [3]:
import subprocess

url = "https://apis.data.go.kr/B551011/GoCamping/basedList?numOfRows=50&pageNo=1&MobileOS=ETC&MobileApp=camping&serviceKey=9CddWKBqyjQQ6afe3WLOVYPvCrXYuxPPd3%2BeA%2F7mVVSdg%2B2%2BZJXGyC%2B0RwPk8wDMSQpJyHc1gXYzC3h4OErgiQ%3D%3D&_type=xml"
result = subprocess.run(["curl","-k", url], capture_output=True, text=True, encoding='utf-8')
print(result.stdout) #표준 아웃..?

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header>
  <resultCode>0000</resultCode>
  <resultMsg>OK</resultMsg>
</header><body><items>
  <item>
    <contentId>101825</contentId>
    <facltNm>루트113 베이스캠프</facltNm>
    <lineIntro></lineIntro>
    <intro></intro>
    <allar>0</allar>
    <insrncAt>Y</insrncAt>
    <trsagntNo></trsagntNo>
    <bizrno></bizrno>
    <facltDivNm></facltDivNm>
    <mangeDivNm>직영</mangeDivNm>
    <mgcDiv></mgcDiv>
    <manageSttus>운영</manageSttus>
    <hvofBgnde></hvofBgnde>
    <hvofEnddle></hvofEnddle>
    <featureNm></featureNm>
    <induty>자동차야영장</induty>
    <lctCl>산,숲</lctCl>
    <doNm>울산광역시</doNm>
    <sigunguNm>울주군</sigunguNm>
    <zipcode>44956</zipcode>
    <addr1>울산 울주군 삼동면 보삼길 115</addr1>
    <addr2></addr2>
    <mapX>129.115575474596</mapX>
    <mapY>35.4896730203951</mapY>
    <direction></direction>
    <tel></tel>
    <homepage></homepage>
    <resveUrl></resveUrl>
    <resveCl></resveCl>
    <manageNmpr>0</manageNmpr>
    

In [17]:
import subprocess
import xml.etree.ElementTree as ET
import requests
import openpyxl
from openpyxl.drawing.image import Image as ExcelImage #이미지를 가져온다
from io import BytesIO # io 입출력 관리하는 라이브러리

url = "https://apis.data.go.kr/B551011/GoCamping/basedList?numOfRows=50&pageNo=1&MobileOS=ETC&MobileApp=camping&serviceKey=9CddWKBqyjQQ6afe3WLOVYPvCrXYuxPPd3%2BeA%2F7mVVSdg%2B2%2BZJXGyC%2B0RwPk8wDMSQpJyHc1gXYzC3h4OErgiQ%3D%3D&_type=xml"

result = subprocess.run(["curl","-k", url], capture_output=True, text=True, encoding='utf-8')
root = ET.fromstring(result.stdout)
#print(result.stdout)

wb = openpyxl.Workbook()
ws = wb.active
ws.title = "캠핑장정보"
ws.append(["아이디", "캠핑장명", "이미지","지역","주소","홈페이지"])

items = root.findall(".//item")
#print(item)
row = 2 # 행은 row


for item in items :
    contentId =item.findtext("contentId",default="")
    facltNm =item.findtext("facltNm",default="")
    firstImageUrl =item.findtext("firstImageUrl",default="")
    doNm =item.findtext("doNm",default="")
    addr1 =item.findtext("addr1",default="")
    homepage =item.findtext("homepage",default="")

    ws.append([contentId, facltNm,"",doNm,addr1,homepage])

    if firstImageUrl :
        try : 
            img_data = requests.get(firstImageUrl, timeout=10).content
            img_file = BytesIO(img_data)
            img = ExcelImage(img_file)
            img.width = 100
            img.height = 75
            ws.add_image(img,f"C{row}")
            ws.row_dimensions[row].height= 50
            
        except Exception as e: 
            print(f"[경고] 이미지 다운로드 실패 : {firstImageUrl} ->{e} ")
    row += 1

ws.column_dimensions["B"].width = 30
ws.column_dimensions["C"].width = 20
ws.column_dimensions["D"].width = 15
ws.column_dimensions["E"].width = 50
ws.column_dimensions["F"].width = 50


wb.save("camping_info_1.xlsx")
print("✅camping_info_1.xlsx 파일 저장 완료")

✅camping_info_1.xlsx 파일 저장 완료
